# Quantize your model for NPU of i.mx 8M Plus

For more information please read here: https://towardsdatascience.com/quantize-your-deep-learning-model-to-run-on-an-npu-2900191757e5

## Import Libraries

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import pathlib as Path

tfVersion=tf.version.VERSION.replace(".", "")# use for later savename
print(tf.version.VERSION)

## Create Representative Dataset

In [ ]:
folderpath='./All_croped_images/'

def prepare(img):
    img = np.expand_dims(img,0).astype(np.float32)
    img = preprocess_input(img, version=2)
    return img
      
repDatagen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=prepare)
datagen=repDatagen.flow_from_directory(folderpath,target_size=(224,224),batch_size=1)

## Load your model here from h5, pb, or import it via Git, or just train the model here.
Example here with a saved h5 model

In [ ]:
#Load the h5 model 
pretrained_model = tf.keras.models.load_model('my_model.h5')

--------------------------------------------------------

## Now set the Converter Depending on your TF Version and how you load the model




### Use a Loaded Keras Model
with any version

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(pretrained_model) # Works only with TF1.x       

### Load frome h5 file
### TF 2.x

In [ ]:
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(saved_model_dir + h5_modelname) #works now also with TF2.x

### TF < 2.x

In [ ]:
#Then use the converter as before
converter = tf.lite.TFLiteConverter.from_keras_model_file(pretrained_model)

### Load from saved model (pb) file

### TF 2.x 
Here we recommend not to use any older version and the save and freeze graph method. The pb files of TF < 1.15 are fundamentally differnet from the ones TF >1.15

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) # the folder contains the pb file and a assets and variables folder

____________________________________________________________

## Now Fully Convert the Model with either in- and output int8 or fLoat32  

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.experimental_new_converter = True

converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.int8 
converter.inference_output_type = tf.int8 
quantized_tflite_model = converter.convert()

open('quant_model.tflite' , "wb").write(quantized_tflite_model)